In [1]:
import pandas as pd
import numpy as np

In [2]:
file1 = pd.read_csv("C:\\Users\\Shyam Mashru\\OneDrive\\Desktop\\SDP_Project\\IPL_Final_Data.csv")
file2 = pd.read_csv("C:\\Users\\Shyam Mashru\\OneDrive\\Desktop\\SDP_Project\\IPL_Ball_By_Ball_Updated.csv")

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Shyam Mashru\\OneDrive\\Desktop\\SDP_Project\\IPL_Final_Data.csv'

In [ ]:
file1

In [ ]:
file2

In [ ]:
total_score = file2.groupby(['ID','innings']).sum()['total_run'].reset_index()
total_score

In [ ]:
total_score_first = total_score[total_score['innings']==1]
total_score_first

In [ ]:
match = pd.merge(file1,total_score_first[['ID','total_run']],left_on='ID',right_on='ID',how='left')
match.rename({'total_run':'FirstInningsRuns'},axis=1,inplace=True)
match.columns

In [ ]:
match = match[['ID', 'Team1', 'Team2', 'City', 'TossWinner','TossDecision',
       'WinningTeam','HomeGround','FirstInningsRuns']]

In [ ]:
delivery = match.merge(file2,on='ID')

In [ ]:
delivery.to_csv("C:\\Users\\Shyam Mashru\\OneDrive\\Desktop\\SDP_Project\\a.csv",index=False)

In [ ]:
delivery_second = delivery[delivery['innings']==2]
delivery_second

In [ ]:
delivery_second['CurrentScore'] = delivery_second.groupby('ID')['total_run'].cumsum()
delivery_second

In [ ]:
delivery_second['RunsLeft'] = delivery_second['FirstInningsRuns'] + 1 - delivery_second['CurrentScore'] 
delivery_second

In [ ]:
delivery_second['BallsLeft'] = 120 - (delivery_second['overs']*6 + delivery_second['ballnumber'])
delivery_second

In [ ]:
delivery_second['isWicketDelivery'] = delivery_second['isWicketDelivery'].astype(int)
wickets = delivery_second.groupby('ID')['isWicketDelivery'].cumsum().values
delivery_second['WicketsLeft'] = 10 - wickets
delivery_second

In [ ]:
delivery_second['CurrentRunRate'] = (delivery_second['CurrentScore']*6)/(120-delivery_second['BallsLeft'])
delivery_second

In [ ]:
delivery_second['RequiredRunRate'] = (delivery_second['RunsLeft']*6)/delivery_second['BallsLeft']
delivery_second

In [ ]:
def result(row):
    return 1 if row['BattingTeam'] == row['WinningTeam'] else 0

In [ ]:
delivery_second['Result'] = delivery_second.apply(result,axis=1)
delivery_second

In [ ]:
delivery_second.columns

In [ ]:
final_second = delivery_second[['City','HomeGround','BattingTeam','BowlingTeam','FirstInningsRuns', 'CurrentScore', 'RunsLeft',
       'BallsLeft', 'WicketsLeft', 'CurrentRunRate',
       'RequiredRunRate', 'Result']]
final_second

In [ ]:
final_second = final_second.sample(final_second.shape[0])

In [ ]:
final_second.sample()

In [ ]:
final_second.dropna(inplace=True)

In [ ]:
final_second = final_second[final_second['BallsLeft']!=0]

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

encode_columns = ['City','BattingTeam','BowlingTeam','HomeGround']
                  
for column in encode_columns:
                  final_second[column] = label_encoder.fit_transform(final_second[column])

final_second

In [ ]:
columns = final_second.columns.tolist()
if 'Result' in columns:
    columns.remove('Result')
columns

In [ ]:
X = final_second.drop(columns=['Result'])
y = final_second['Result']

In [ ]:
 from sklearn.model_selection import train_test_split
 X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4,random_state=10000)

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
model = LogisticRegression()
#model = RandomForestClassifier()
#model = DecisionTreeClassifier(random_state=42)
#model = SVC(kernel='linear', C=1)
#model = XGBClassifier()
#model = GaussianNB()
#model = SVC(kernel='rbf', random_state=0)
#model = KNeighborsClassifier(n_neighbors=5,metric='minkowski',p=2)

In [ ]:
model.fit(X_train,y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
accuracy_score(y_test,y_pred)

In [ ]:
conf_matrix = confusion_matrix(y_test, y_pred)
classification_rep = classification_report(y_test, y_pred)

In [ ]:
print('Confusion Matrix:')
print(conf_matrix)
print('Classification Report:')
print(classification_rep)

In [ ]:
import pickle
pickle.dump(model,open('C:\\Users\\Shyam Mashru\\OneDrive\\Desktop\\SDP_Project\\pipe2.pkl','wb'))

In [ ]:
model.predict_proba(X_test)[22]

In [ ]:
instance_26 = X_test.iloc[22]
print(instance_26)